The following is the 1-D Linear Convection equation:
$$
\begin{equation*}
    \frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0.
\end{equation*}
$$
This PDE describes the propagation of a wave in one dimension. Let $u_0(x) = u(x, 0)$ describe the initial condition of the equation. The solution to the equation above is $u(x, t) = u_0(x - ct)$. We will try to see how we can recover this through time and space discretization.

First, we split 

In [1]:
from plotly.offline import init_notebook_mode
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import time, sys

init_notebook_mode(connected=True)

In [2]:
def convection_simulation(u0, Nx, Nt, dt, c=1, boundary_val=1):
    dx = 2/(Nx - 1)
    u = np.zeros((Nt + 1, Nx))

    u[:] = boundary_val
    u[0] = u0

    for t in range(1, Nt + 1):
        u[t][1:] = u[t - 1][1:] - c * dt / dx * (u[t - 1][1:] - u[t - 1][:Nx - 1])
    
    return u


def square_wave_convection(Nx, Nt, dt, c=1, low=1, high=2):
    dx = 2/(Nx - 1)
    u0 = np.zeros(Nx)

    u0[:] = low
    u0[int(.5/dx):int(1/dx + 1)]  = high

    return convection_simulation(u0, Nx, Nt, dt, c)

def make_convection_animation(u, Nx, Nt, min_x, max_x, transition_time=0, frame_time=100):
    u_df = pd.DataFrame({
        "x": np.linspace(min_x, max_x, Nx),
    })

    for i in range(Nt + 1):
        u_df[str(i)] = u[i]

    fig = px.line(u_df.melt("x"), x="x", y="value", animation_frame="variable")
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = transition_time
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = frame_time

    return fig

    


In [3]:
u = square_wave_convection(41, 25, 0.025)

make_convection_animation(u, 41, 25, 0, 2)

In [4]:
u = square_wave_convection(81, 25, 0.025)

make_convection_animation(u, 81, 25, 0, 2)

Why do we have such a different behavior? When $N_x = 81$, we have $\Delta x = 0.025$ and thus our transition equation
$$
\begin{equation*}
    u_i^{n+1} = u_i^n - c \frac{\Delta t}{\Delta x}\left(u_i^n - u_{i-1}^n\right)
\end{equation*}
$$
becomes
$$
\begin{equation*}
    u_i^{n+1} = u_i^n - \left(u_i^n - u_{i-1}^n\right) = u_{i-1}^n,
\end{equation*}
$$
which is reminiscent of the analytical solution to the original equation. This happens whenever $c\cdot\frac{\Delta t}{\Delta x} = 1$.

In [6]:
u = square_wave_convection(121, 25, 0.025)
make_convection_animation(u, 121, 25, 0, 2)